# 【第3回】AIエンジニアリング実践 任意課題

##宿題の概要
この宿題では、講義で学んだRAG（Retrieval-Augmented Generation）技術を用いて、LLMの生成内容を改善する実践的な取り組みを行います。演習で利用したコードをベースに、独自の質問と参照文書を用いて実験を行い、RAGの効果を定量的・定性的に評価します。

この宿題を通じて、「テストデータの作成」と「改善のプロセス」について理解を深め、実際のアプリケーション開発
に役立てることを目指します。

##宿題の内容
1. **独自の質問と参照資料の作成**  
 * 自分で5つ以上の質問文を考案してください
 * 各質問に対する回答を含む参照文書を用意してください
 * 少なくとも1つは、LLMが単体では正確に答えられないような知識を含む質問にしてください


2. **実験の実施**
 * 演習で使用したコードをベースに、以下の2つの方法で回答を生成してください
    * ベースのLLM（RAGなし）での回答生成  
    * RAGを組み合わせた回答生成  
 * 回答の評価では、単純なYes/No判定でも良いです  
    * より詳細な評価指標も検討していただけるとなお良いです  
3. **結果分析と考察**
 * 生成した結果をまとめ、RAGありとRAGなしの差異を分析してください
 * RAGによって回答が改善したケースと悪化したケースの両方について考察してください
 * 結果に基づいて、RAGの有効性と限界についての考察を記述してください


## 扱う質問

以下、2025年3月に発見された tj-actions/changed-files の脆弱性（CVE-2025-30066）をテーマにした質問例を 6 つ挙げる。（モデル単体ではリアルタイムに正確に答えにくい最新情報を含む。）
1. **脆弱性の発生タイムライン**  
   tj-actions/changed-files リポジトリにおいて、悪意あるペイロードが最初にコミットされたのはいつか、またそれが公表されたのはいつかを教えてください。
2. **攻撃手法と影響範囲**  
  この脆弱性によって GitHub Actions のログに漏洩した具体的な秘密情報（環境変数や認証トークンなど）の例を挙げてください。
3. **検出・診断方法**  
  既存のリポジトリが影響を受けているかを自動でチェックするために有効なスクリプトやクエリの例を示してください。
4. **修正内容の技術的詳細**  
  悪意あるコードがどのファイル／関数に追加されていたかを示し、修正パッチではどのようにコードが書き換えられたかを説明してください。
5. **公式アドバイザリとドキュメント**  
   GitHub Security Advisory や CISA アラートで公開された、この脆弱性に関する公式ドキュメントの識別子（ID）や URL を教えてください。
6. **影響を受けた組織・プロジェクト**  
   実際に被害が報告された主要なオープンソースプロジェクト名や企業名、およびそれらが公開したインシデント報告のリンクを挙げてください。


## 回答を含む参照文書

1. **脆弱性の発生タイムライン**  
  脆弱性発生タイムラインは、2025年3月12日から15日にかけてtj-actions/changed-filesのタグが悪意あるコミットに差し替えられ、3月14日 StepSecurityが異常検知、3月15日 GitHubがリポジトリを一時非公開化し3月17日にv46.0.1がリリースされた。
2. **撃手法と影響範囲**  
  攻撃手法と漏洩情報は、index.jsに外部Gistからmemdump.pyをダウンロード・実行するコードが注入され、RunnerのメモリをダンプしてAWSキー、GitHub PAT、npmトークン、RSA鍵などをログに書き出した。
3. **検出・診断方法**  
  検出・診断方法は、GitHubコード検索でtj-actions/changed-filesの使用箇所を特定し、Falco ActionsやHarden-Runnerで外部接続を監視し、ログからBase64二重エンコード文字列を抽出してシークレットパターンを検査する。
4. **修正内容の技術的詳細**  
  修正内容は、悪意あるGist取得・実行処理をindex.jsから削除し、action.ymlで外部スクリプト読み込みを禁止、全タグを安全なコミットに再ポイントしてv46.0.1以降をリリースした。
5. **公式アドバイザリとドキュメント**  
  公式アドバイザリは、CVE-2025-30066、GHSA-MRRH-FWG8-R2C3、CISAアラート「Supply Chain Compromise of Third-Party tj-actions/changed-files」（2025-03-18公開）。
6. **影響を受けた組織・プロジェクト**  
  影響を受けた組織は、espressif/arduino-esp32、chains-project/maven-lockfile、rackerlabs/genestack、modal-labs/modal-examplesなど約23,000リポジトリが使用し、公開ログ保持プロジェクトで漏洩が多発し、StepSecurityやAqua Securityが詳細レポートを公開。


## 扱うモデル

「google/gemma-2-2b-jpn-it」を使用します。このモデルは、リリース時期の関係上、以下の特徴を持ちます。

- tj-actions/changed-files の脆弱性（CVE-2025-30066）情報が広まる前に訓練されており、このトピックに関する知識を持たないと想定される
- この特性を活かし、純粋なベースライン評価から各手法の効果を観察する

## 演習環境の準備

In [1]:
!pip install --upgrade transformers
!pip install google-colab-selenium
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
# 演習用のコンテンツを取得
!git clone https://github.com/taitai-2009/lecture-ai-engineering.git

Cloning into 'lecture-ai-engineering'...
remote: Enumerating objects: 86, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 86 (delta 8), reused 14 (delta 6), pack-reused 69 (from 1)
Receiving objects: 100% (86/86), 111.55 KiB | 15.94 MiB/s, done.
Resolving deltas: 100% (25/25), done.


In [3]:
# HuggingFace Login
from huggingface_hub import notebook_login

notebook_login()

In [4]:
# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
import random
random.seed(0)

In [6]:
# モデル(Gemma2)の読み込み

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "google/gemma-2-2b-jpn-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
        )

tokenizer_config.json:   0%|          | 0.00/46.9k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

# 1. ベースラインモデル評価
**まずはベースモデルがどの程度知識を持っているか確かめる**

In [77]:
def generate_output(query):
  messages = [
      {"role": "user", "content": query},
  ]
  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      #max_new_tokens=256,
      max_new_tokens=512,
      eos_token_id=terminators,
      do_sample=False,
      # temperature=0.6, # If do_sample=True
      # top_p=0.9,  # If do_sample=True
  )

  response = outputs[0][input_ids.shape[-1]:]
  return tokenizer.decode(response, skip_special_tokens=True)

In [78]:
#question =  "LLMにおけるInference Time Scalingとは？"
question =  "tj-actions/changed-files の脆弱性によって、悪意あるコードがどのファイル／関数に追加されていたかを示し、この脆弱性により、どのような影響があったか、また、修正パッチではどのようにコードが書き換えられたかを説明してください。"
response = generate_output(question)
print(response)

## tj-actions/changed-files の脆弱性に関する説明

**脆弱性概要:**

tj-actions/changed-files にある脆弱性により、悪意のあるコードが特定のファイルや関数のみに追加されていた可能性があります。具体的な情報については、脆弱性に関する公開情報やセキュリティ報告書を参照してください。

**影響:**

悪意のあるコードの追加によって、以下の様な影響が発生する可能性があります。

* **データ漏洩:**  悪意のあるコードが、ユーザーのデータや機密情報にアクセスできるように変更された場合。
* **システムの悪用:**  悪意のあるコードが、システムの機能を悪用して、不正な操作を行う可能性がある。
* **システムの破損:**  悪意のあるコードが、システムの正常な動作を妨害したり、システムを破壊する可能性がある。
* **セキュリティの脆弱性:**  悪意のあるコードが、システムのセキュリティを弱体化したり、攻撃に対して脆弱な状態にする可能性がある。

**修正パッチ:**

修正パッチでは、脆弱性によって影響を受けたファイルや関数のコードが書き換えられました。具体的な変更内容については、公開情報やセキュリティ報告書を参照してください。

**注意:**

* TJ-actions/changed-files の脆弱性に関する情報は、公開情報やセキュリティ報告書を参考にする必要があります。
* 具体的な情報が公開されていない場合は、セキュリティ専門家や開発チームに相談してください。



 





- 演習のサンプルに従い、数値的な評価も見てみます。RagasにはAnswer Accuracyという評価指標があります。今回はこちらを参考に実装した評価関数を利用して測っていきます。

- 演習のサンプルに従い、OpenAIのgpt-4oで評価していきます。

In [9]:
!pip install -U openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.0/662.0 kB 32.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.76.2
    Uninstalling openai-1.76.2:
      Successfully uninstalled openai-1.76.2


In [79]:
# @title 評価実装
#gold_answer = "「Inference Time Scaling」とは、推論時に計算量を増やしてモデルの性能を高める手法です。これはモデルのサイズを大きくする代わりに、難しい入力に対して多くの計算リソースを使うことで、より良い出力を得ようとするアプローチです。"
gold_answer = "tj-actions/changed-files 脆弱性の影響は、GitHub Runner のメモリをダンプし、環境変数やプロセス中のシークレットを抽出してビルドログに出力した。この時、AWSキー、GitHub PAT、npmトークン、RSA鍵などをビルドログに書き出した。修正内容は、悪意あるGist取得・実行処理をindex.jsから削除し、action.ymlで外部スクリプト読み込みを禁止、全タグを安全なコミットに再ポイントしてv46.0.1以降をリリースしています。"

from openai import OpenAI
from google.colab import userdata
client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"), max_retries=5, timeout=60)

def openai_generator(query):

        messages = [
                    {
                        "role": "user",
                        "content": query
                    }
                ]

        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages
        )
        return response.choices[0].message.content

def evaluate_answer_accuracy(query, response, reference):

    template_accuracy1 = (
          "Instruction: You are a world class state of the art assistant for rating "
          "a User Answer given a Question. The Question is completely answered by the Reference Answer.\n"
          "Say 4, if User Answer is full contained and equivalent to Reference Answer"
          "in all terms, topics, numbers, metrics, dates and units.\n"
          "Say 2, if User Answer is partially contained and almost equivalent to Reference Answer"
          "in all terms, topics, numbers, metrics, dates and units.\n"
          "Say 0, if User Answer is not contained in Reference Answer or not accurate in all terms, topics,"
          "numbers, metrics, dates and units or the User Answer do not answer the question.\n"
          "Do not explain or justify your rating. Your rating must be only 4, 2 or 0 according to the instructions above.\n"
          "Even small discrepancies in meaning, terminology, directionality, or implication must result in a lower score. Only rate 4 if the User Answer is a complete and precise match to the Reference Answer in every aspect.\n"
          "### Question: {query}\n"
          "### {answer0}: {sentence_inference}\n"
          "### {answer1}: {sentence_true}\n"
          "The rating is:\n"
      )
    template_accuracy2 = (
          "I will rate the User Answer in comparison to the Reference Answer for a given Question.\n"
          "A rating of 4 indicates that the User Answer is entirely consistent with the Reference Answer, covering all aspects, topics, numbers, metrics, dates, and units.\n"
          "A rating of 2 signifies that the User Answer is mostly aligned with the Reference Answer, with minor discrepancies in some areas.\n"
          "A rating of 0 means that the User Answer is either inaccurate, incomplete, or unrelated to the Reference Answer, or it fails to address the Question.\n"
          "I will provide the rating without any explanation or justification, adhering to the following scale: 0 (no match), 2 (partial match), 4 (exact match).\n"
          "Even minor inconsistencies in meaning, terminology, emphasis, or factual detail should prevent a rating of 4. Only assign a 4 if the User Answer exactly and unambiguously matches the Reference Answer in every respect."
          "Do not explain or justify my rating. My rating must be only 4, 2 or 0 only.\n\n"
          "Question: {query}\n\n"
          "{answer0}: {sentence_inference}\n\n"
          "{answer1}: {sentence_true}\n\n"
          "Rating: "
      )

    score1 = openai_generator(
                template_accuracy1.format(
                      query=query,
                      answer0="User Answer",
                      answer1="Reference Answer",
                      sentence_inference=response,
                      sentence_true=reference,
                    )
                )
    try:
      score1 = int(score1)
    except:
      print("Failed")
      score1 = 0

    score2 = openai_generator(
                template_accuracy2.format(
                        query=query,
                        answer0="Reference Answer",
                        answer1="User Answer",
                        sentence_inference=reference,
                        sentence_true=response,
                    )
                  )

    try:
      score2 = int(score2)
    except:
      print("Failed")
      score2 = 0


    return (score1 + score2) / 2

In [80]:
# 評価
score = evaluate_answer_accuracy(question, response, gold_answer)
print(score)

0.0


## 結果 (ベースモデル)

「google/gemma-2-2b-jpn-it」は「tj-actions/changed-files の脆弱性」について誤った知識を提示しました：
* モデルは脆弱性のにより影響を受けたファイルについて具体的な事例がなく、修正ついての回答が一般的な内容となっている。

---

# 2. 回答・参照情報データの活用

## 2.1 回答・参照情報データをソースとして活用 (RAG導入)

モデルの回答の事実性を向上させるためにRetrieval Augmented Generation (RAG)技術を導入します：

* **知識ソース**: 回答・参照情報データ（tj-actions-raw.txt）
* **目的**: モデルに「tj-actions/changed-files の脆弱性」に関する正確な知識と文脈を提供し、事実に基づいた回答を促す

**初期RAG実装（ベーシックアプローチ）**:
* **ドキュメント処理**: 回答・参照情報データを含む生テキストをそのまま使用
* **分割方法**: 「。」（句点）で区切られた文単位でテキストを分割
* **検索手法**: シンプルな類似度ベースの検索でクエリに関連する文を抽出
* **制約条件**: モデルの入力トークン制限に収まるよう関連文のみを選択

In [12]:
from sentence_transformers import SentenceTransformer

emb_model = SentenceTransformer("infly/inf-retriever-v1-1.5b", trust_remote_code=True)
# In case you want to reduce the maximum length:
emb_model.max_seq_length = 4096

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/284 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/19.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/918 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

### 回答を含む参照情報の生データの表示と取り込み



In [81]:
# 回答を含む参照情報の生データの表示
!cat /content/lecture-ai-engineering/day3/data/tj-actions-raw.txt

脆弱性発生タイムラインは、2025年3月12日から15日にかけてtj-actions/changed-filesのタグが悪意あるコミットに差し替えられ、3月14日 StepSecurityが異常検知、3月15日 GitHubがリポジトリを一時非公開化し3月17日にv46.0.1がリリースされた。
攻撃手法と漏洩情報は、index.jsに外部Gistからmemdump.pyをダウンロード・実行するコードが注入され、RunnerのメモリをダンプしてAWSキー、GitHub PAT、npmトークン、RSA鍵などをログに書き出した。
検出・診断方法は、GitHubコード検索でtj-actions/changed-filesの使用箇所を特定し、Falco ActionsやHarden-Runnerで外部接続を監視し、ログからBase64二重エンコード文字列を抽出してシークレットパターンを検査する。
修正内容は、悪意あるGist取得・実行処理をindex.jsから削除し、action.ymlで外部スクリプト読み込みを禁止、全タグを安全なコミットに再ポイントしてv46.0.1以降をリリースした。
公式アドバイザリは、CVE-2025-30066、GHSA-MRRH-FWG8-R2C3、CISAアラート「Supply Chain Compromise of Third-Party tj-actions/changed-files」（2025-03-18公開）。
影響を受けた組織は、espressif/arduino-esp32、chains-project/maven-lockfile、rackerlabs/genestack、modal-labs/modal-examplesなど約23,000リポジトリが使用し、公開ログ保持プロジェクトで漏洩が多発し、StepSecurityやAqua Securityが詳細レポートを公開。

In [82]:
#with open("/content/lecture-ai-engineering/day3/data/LLM2024_day4_raw.txt", "r") as f:
with open("/content/lecture-ai-engineering/day3/data/tj-actions-raw.txt", "r") as f:
  raw_writedown = f.read()

In [83]:
# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[3])

ドキュメントサイズ:  7
ドキュメントの例: 
 修正内容は、悪意あるGist取得・実行処理をindex.jsから削除し、action.ymlで外部スクリプト読み込みを禁止、全タグを安全なコミットに再ポイントしてv46.0.1以降をリリースした


In [84]:
# Retrievalの実行
#question = "LLMにおけるInference Time Scalingとは？"
question = "tj-actions/changed-files の脆弱性によって、悪意あるコードがどのファイル／関数に追加されていたかを示し、この脆弱性により、どのような影響があったか、また、修正パッチではどのようにコードが書き換えられたかを説明してください。"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[76.04584503173828, 68.49787902832031, 73.98768615722656, 72.06385040283203, 70.93110656738281, 66.7538833618164, 54.41156005859375]]


In [85]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

取得したドキュメント1: (Score: 76.04584503173828)
脆弱性発生タイムラインは、2025年3月12日から15日にかけてtj-actions/changed-filesのタグが悪意あるコミットに差し替えられ、3月14日 StepSecurityが異常検知、3月15日 GitHubがリポジトリを一時非公開化し3月17日にv46.0.1がリリースされた 


取得したドキュメント2: (Score: 73.98768615722656)
検出・診断方法は、GitHubコード検索でtj-actions/changed-filesの使用箇所を特定し、Falco ActionsやHarden-Runnerで外部接続を監視し、ログからBase64二重エンコード文字列を抽出してシークレットパターンを検査する 


取得したドキュメント3: (Score: 72.06385040283203)
修正内容は、悪意あるGist取得・実行処理をindex.jsから削除し、action.ymlで外部スクリプト読み込みを禁止、全タグを安全なコミットに再ポイントしてv46.0.1以降をリリースした 


取得したドキュメント4: (Score: 70.93110656738281)
公式アドバイザリは、CVE-2025-30066、GHSA-MRRH-FWG8-R2C3、CISAアラート「Supply Chain Compromise of Third-Party tj-actions/changed-files」（2025-03-18公開） 


取得したドキュメント5: (Score: 68.49787902832031)
攻撃手法と漏洩情報は、index.jsに外部Gistからmemdump.pyをダウンロード・実行するコードが注入され、RunnerのメモリをダンプしてAWSキー、GitHub PAT、npmトークン、RSA鍵などをログに書き出した 




In [86]:
references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
#query =  f"[参考資料]\n{references}\n\n[質問] LLMにおけるInference Time Scalingとは？"
query =  f"[参考資料]\n{references}\n\n[質問] tj-actions/changed-files の脆弱性によって、悪意あるコードがどのファイル／関数に追加されていたかを示し、この脆弱性により、どのような影響があったか、また、修正パッチではどのようにコードが書き換えられたかを説明してください。"
response = generate_output(query)
print(response)

## tj-actions/changed-files 脆弱性に関する詳細

**脆弱性内容:**

* **悪意あるGist取得・実行処理の注入:**  `index.js` ファイルに外部Gistから`memdump.py`をダウンロード・実行するコードが注入されました。
* **Runnerのメモリダンプ:**  `memdump.py`実行により、Runnerのメモリをダンプし、AWSキー、GitHub PAT、npmトークン、RSA鍵などをログに書き出しました。

**影響:**

* **機密情報の漏洩:**  Runnerのメモリから取得された情報が、攻撃者によって悪用される可能性があります。
* **システムへの侵入:**  `memdump.py`は、攻撃者によるシステムへの侵入を可能にする可能性があります。
* **アカウント情報へのアクセス:**  `npmトークン`や`GitHub PAT`などの情報が漏洩することで、攻撃者によるアカウントへのアクセスが容易になります。



**修正内容:**

* **`index.js` ファイルの削除:**  悪意あるGist取得・実行処理を削除しました。
* **`action.yml` ファイルの変更:**  外部スクリプト読み込みを禁止しました。
* **全タグの安全なコミットへの再ポイント:**  v46.0.1以降をリリースしました。
* **安全なコミットへの移行:**  全タグを安全なコミットに再ポイントすることで、脆弱性による影響を最小限に抑えました。



**まとめ:**

tj-actions/changed-files の脆弱性により、悪意のあるコードが `index.js` ファイルに注入され、Runnerのメモリをダンプして機密情報が漏洩する可能性がありました。修正パッチでは、`index.js` ファイルの悪意ある処理を削除し、外部スクリプト読み込みを禁止することで、攻撃の危険性を抑えました。 
 





In [87]:
# 評価
score = evaluate_answer_accuracy(question, response, gold_answer)
print(score)

3.0


### 結果 (初期RAG実装)

回答・参照情報のファイルにある情報を元に、回答の生成ができた。スコアは悪くないが、具体的な影響（何のログへの出力なのか）がわからない。

### 問題分析
以下の要因が考えられます：
1. **回答精度の課題**: 脆弱性の悪用により、AWSキー、GitHub PAT、npmトークン、RSA鍵などが何のログに書き出されたのかが明確でない。

## 2.2 回答・参照情報データの品質改善

攻撃手法と漏洩情報について、もう少し詳しく書いたデータを用意する。

In [88]:
# 回答を含む参照情報の生データ（改良版）の表示
!cat /content/lecture-ai-engineering/day3/data/tj-actions.txt

脆弱性発生タイムラインは、2025年3月12日から15日にかけて tj-actions/changed-files のタグが全て悪意あるコミットに差し替えられ、回帰的に改ざんされたことがCISAからのアラートとして発出。3月14日17:00UTC StepSecurityが異常検知、問題の調査を開始。3月14日20:30UTC 複数の公開リポジトリでシークレット漏洩が確認されていることを、Aqua Security がブログで速報。3月15日 GitHubが影響を受けた Action レポジトリを一時的に非公開化し、3月17日に悪意あるコードが除去されたパッチバージョン v46.0.1 がリリースされた。
攻撃手法と漏洩情報は、攻撃者は index.js に Node.js で書かれた関数を注入し、外部 Gist から memdump.py をダウンロード・実行。GitHub Runner のメモリをダンプし、環境変数やプロセス中のシークレットを抽出してビルドログに出力した。この時、AWSキー、GitHub PAT、npmトークン、RSA鍵などをビルドログに書き出した。
検出・診断方法は、GitHubコード検索で tj-actions/changed-files の使用箇所を特定し、Falco ActionsやHarden-Runnerで外部接続を監視し、CI/CD ワークフローで gist.githubusercontent.com への不審な外部接続をリアルタイム検出するモジュールを導入する。ログからBase64二重エンコード文字列を抽出して、出力内に AKIA… や ghp_… などのシークレットパターンが含まれていないかを検査する。
修正内容は、悪意あるGist取得・実行処理をindex.jsから削除し、action.ymlで外部スクリプト読み込みを禁止、全タグを安全なコミットに再ポイントしてv46.0.1以降をリリースした。
公式アドバイザリは、CVE-2025-30066、CVE-2025-30154、GHSA-MRRH-FWG8-R2C3、CISAアラート「Supply Chain Compromise of Third-Party tj-actions/changed-files」（2025-03-18公開）。
影響範囲は、23,000 以上のリポジ

In [89]:
#with open("/content/lecture-ai-engineering/day3/data/LLM2024_day4.txt", "r") as f:
with open("/content/lecture-ai-engineering/day3/data/tj-actions.txt", "r") as f:

  raw_writedown = f.read()

In [90]:
# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[6])

ドキュメントサイズ:  14
ドキュメントの例: 
 この時、AWSキー、GitHub PAT、npmトークン、RSA鍵などをビルドログに書き出した


In [91]:
# Retrievalの実行
#question = "LLMにおけるInference Time Scalingとは？"
question = "tj-actions/changed-files の脆弱性によって、悪意あるコードがどのファイル／関数に追加されていたかを示し、この脆弱性により、どのような影響があったか、また、修正パッチではどのようにコードが書き換えられたかを説明してください。"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[77.56555938720703, 63.30323791503906, 65.41535949707031, 71.5719985961914, 66.8084487915039, 61.46861267089844, 63.04637145996094, 71.39595031738281, 62.657737731933594, 72.06388092041016, 70.05047607421875, 77.0645980834961, 67.0576400756836, 54.41155242919922]]


In [92]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

取得したドキュメント1: (Score: 77.56555938720703)
脆弱性発生タイムラインは、2025年3月12日から15日にかけて tj-actions/changed-files のタグが全て悪意あるコミットに差し替えられ、回帰的に改ざんされたことがCISAからのアラートとして発出 


取得したドキュメント2: (Score: 77.0645980834961)
影響範囲は、23,000 以上のリポジトリで tj-actions/changed-files が使用されており、公開ビルドログを持つプロジェクトは特にリスクが高い 


取得したドキュメント3: (Score: 72.06388092041016)
修正内容は、悪意あるGist取得・実行処理をindex.jsから削除し、action.ymlで外部スクリプト読み込みを禁止、全タグを安全なコミットに再ポイントしてv46.0.1以降をリリースした 


取得したドキュメント4: (Score: 71.5719985961914)
3月15日 GitHubが影響を受けた Action レポジトリを一時的に非公開化し、3月17日に悪意あるコードが除去されたパッチバージョン v46.0.1 がリリースされた 


取得したドキュメント5: (Score: 71.39595031738281)
検出・診断方法は、GitHubコード検索で tj-actions/changed-files の使用箇所を特定し、Falco ActionsやHarden-Runnerで外部接続を監視し、CI/CD ワークフローで gist.githubusercontent.com への不審な外部接続をリアルタイム検出するモジュールを導入する 




In [93]:
references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
#query =  f"[参考資料]\n{references}\n\n[質問] LLMにおけるInference Time Scalingとは？"
query =  f"[参考資料]\n{references}\n\n[質問] tj-actions/changed-files の脆弱性によって、悪意あるコードがどのファイル／関数に追加されていたかを示し、この脆弱性により、どのような影響があったか、また、修正パッチではどのようにコードが書き換えられたかを説明してください。"
response = generate_output(query)
print(response)

## tj-actions/changed-files 脆弱性に関する詳細

**脆弱性内容:**

* **悪意あるGist取得・実行処理の追加:** tj-actions/changed-files には、悪意のあるGistを取得・実行する処理が含まれていた。
* **影響範囲:**  23,000 以上のリポジトリで tj-actions/changed-files が使用されており、公開ビルドログを持つプロジェクトは特にリスクが高い。

**影響:**

* **Gistの悪意ある実行:**  悪意のあるGistが実行され、システムに悪影響を及ぼす可能性があった。
* **外部スクリプトの利用制限:**  action.yml で外部スクリプト読み込みを禁止することで、悪意のあるコードの利用を防ぐ。
* **公開ビルドログの脆弱性:** 公開ビルドログを持つプロジェクトでは、悪意のあるコードの公開が容易になり、セキュリティリスクが高まった。

**修正内容:**

* **index.js から悪意ある処理削除:**  悪意のあるGist取得・実行処理をindex.jsから削除。
* **action.yml で外部スクリプト読み込み禁止:**  action.yml で外部スクリプト読み込みを禁止。
* **全タグを安全なコミットに再ポイント:**  v46.0.1以降をリリースし、全タグを安全なコミットに再ポイント。
* **v46.0.1以降のリリース:**  v46.0.1以降をリリースし、悪意のあるコードが削除された。
* **GitHubコード検索で tj-actions/changed-files の使用箇所を特定:**  GitHubコード検索で tj-actions/changed-files の使用箇所を特定。
* **Falco ActionsやHarden-Runnerで外部接続監視:**  Falco ActionsやHarden-Runnerで外部接続を監視。
* **CI/CD ワークフローで gist.githubusercontent.com への不審な外部接続をリアルタイム検出:**  CI/CD ワークフローで gist.githubusercontent.com への不審な外部接続をリアルタイム検出するモジュールを導入

In [94]:
# 評価
score = evaluate_answer_accuracy(question, response, gold_answer)
print(score)

1.0


## 結果 (修正テキストによるRAG)

テキストの品質改善により、モデルの回答に部分的な向上が見られました：

### 改善点
* これまでただのログへの出力との記載だったが、公開されているビルドログへの影響が回答された。

### 問題点
* 漏洩した認証情報についての回答がない。
* 評価スコアが下がった。

### 問題分析

モデルが誤った回答を生成する主要因として、**文脈の欠如**が考えられます：
* 「。」で区切られた短い文単位での検索では、各文の発言背景や関連性が失われる
* 結果として、正しい個別の文でも、その解釈に必要な背景情報が欠如し、誤った文脈で理解される


---



# 3. 文脈を考慮したチャンク化の導入

検索結果の品質向上のため、以下の改善を実施します：

* **前後文脈を含むチャンク化**:
  - 検索でマッチした文だけでなく、その前後の複数文も含めてチャンクとして取得
  - 具体的には、マッチした文を中心に前2文、後2文を含む計5文程度のチャンクを構成
  - この「文脈ウィンドウ」により、発言の背景情報や議論の流れが保持される

* **期待される効果**:
  - 講師の主張とその根拠の関係性を正確に把握できる
  - 概念の定義とその適用範囲を正しく理解できる

この改善により、モデルが講義内容の本質をより正確に理解し、一貫性のある事実に基づいた回答を生成することが期待されます。

In [97]:
# 前後それぞれ2つずつの文章を一つのドキュメントに追加する。（要は5つの文章集合になる)
references = "\n".join(["* " + "。".join(documents[max(0, i-2): min(i+2, len(documents))]).strip() for i in scores.argsort()[0][::-1][:topk]])
#references = "\n".join(["* " + "。".join(documents[max(0, i-3): min(i+3, len(documents))]).strip() for i in scores.argsort()[0][::-1][:topk]])
#query =  f"[参考資料]\n{references}\n\n[質問] LLMにおけるInference Time Scalingとは？"
query =  f"[参考資料]\n{references}\n\n[質問] tj-actions/changed-files の脆弱性によって、悪意あるコードがどのファイル／関数に追加されていたかを示し、この脆弱性により、どのような影響があったか、また、修正パッチではどのようにコードが書き換えられたかを説明してください。"
response = generate_output(query)
print(response)

## tj-actions/changed-files の脆弱性に関する詳細

**脆弱性内容:**

* **悪意あるGist取得・実行処理の追加:**  `index.js` から削除されず、外部スクリプト読み込みを禁止された代わりに、`action.yml` で外部スクリプト読み込みを禁止しました。
* **memdump.py のダウンロード・実行:**  攻撃者は `index.js` に Node.js で書かれた関数を注入し、外部 Gist から `memdump.py` をダウンロード・実行しました。
* **メモリダンプとシークレット情報の抽出:**  GitHub Runner のメモリをダンプし、環境変数やプロセス中のシークレット情報を抽出してビルドログに出力しました。

**影響:**

* **公開リポジトリへの影響:**  約23,000リポジトリが使用しており、公開ビルドログを持つプロジェクトは特にリスクが高い。
* **シークレット情報の漏洩:**  攻撃者は `memdump.py` を実行することで、AWSキー、GitHub PAT、npmトークン、RSA鍵などをビルドログに書き出した。
* **攻撃手法:**  攻撃者は `index.js` に Node.js で書かれた関数を注入し、外部 Gist から `memdump.py` をダウンロード・実行しました。

**修正内容:**

* **悪意あるGist取得・実行処理の削除:**  `index.js` から削除し、`action.yml` で外部スクリプト読み込みを禁止しました。
* **外部スクリプト読み込み禁止:**  `action.yml` で外部スクリプト読み込みを禁止しました。
* **全タグを安全なコミットに再ポイント:**  v46.0.1以降をリリースしました。
* **シークレットパターン検査:**  ログからBase64二重エンコード文字列を抽出して、出力内に AKIA… や ghp_… などのシークレットパターンが含まれていないかを検査しました。



**まとめ:**

tj-actions/changed-files の脆弱性により、悪意のあるコードが `index.js` から削除されず、外部スクリプト読み込みを禁止された代わりに、`ac

In [96]:
# 評価
score = evaluate_answer_accuracy(question, response, gold_answer)
print(score)

2.0


### チャンク数を増やす
マッチした文を中心に前3文、後3文を含む計7文程度のチャンクを構成してみる。



In [98]:
# 前後それぞれ3つずつの文章を一つのドキュメントに追加する。（要は7つの文章集合になる)
#references = "\n".join(["* " + "。".join(documents[max(0, i-2): min(i+2, len(documents))]).strip() for i in scores.argsort()[0][::-1][:topk]])
references = "\n".join(["* " + "。".join(documents[max(0, i-3): min(i+3, len(documents))]).strip() for i in scores.argsort()[0][::-1][:topk]])
#query =  f"[参考資料]\n{references}\n\n[質問] LLMにおけるInference Time Scalingとは？"
query =  f"[参考資料]\n{references}\n\n[質問] tj-actions/changed-files の脆弱性によって、悪意あるコードがどのファイル／関数に追加されていたかを示し、この脆弱性により、どのような影響があったか、また、修正パッチではどのようにコードが書き換えられたかを説明してください。"
response = generate_output(query)
print(response)

## tj-actions/changed-files の脆弱性に関する詳細

**脆弱性内容:**

* **悪意あるコミットへの変更:** tj-actions/changed-files のタグが全て悪意あるコミットに差し替えられ、回帰的に改ざんされた。
* **攻撃手法:**  攻撃者は index.js に Node.js で書かれた関数を注入し、外部 Gist から memdump.py をダウンロード・実行。GitHub Runner のメモリをダンプし、環境変数やプロセス中のシークレットを抽出してビルドログに出力した。
* **影響:**
    * **シークレット漏洩:**  複数の公開リポジトリでシークレットが漏洩した。
    * **ビルドログへの悪意ある情報出力:**  攻撃者はビルドログに環境変数やプロセス中のシークレットを記録した。
    * **外部スクリプト読み込み禁止:**  action.yml で外部スクリプト読み込みを禁止し、安全なコミットに再ポイントした。
    * **悪意あるGist取得・実行処理削除:**  index.js から削除し、安全なコミットに再ポイントした。

**ファイル／関数の詳細:**

* **index.js:**  悪意あるGist取得・実行処理が実行されていた。
* **memdump.py:**  外部 Gist からダウンロード・実行された。

**修正内容:**

* **悪意あるGist取得・実行処理削除:**  index.js から削除。
* **外部スクリプト読み込み禁止:**  action.yml で外部スクリプト読み込みを禁止。
* **全タグを安全なコミットに再ポイント:**  v46.0.1以降をリリース。
* **ビルドログへの書き込み制限:**  AWSキー、GitHub PAT、npmトークン、RSA鍵などをビルドログに書き出した。
* **外部接続監視:**  Falco ActionsやHarden-Runnerで外部接続を監視するモジュールを導入。
* **ログからBase64二重エンコード文字列を抽出して、出力内に AKIA… や ghp_… などのシークレットパターンが含まれていないかを検査する。



**まとめ:**

tj-actions/

In [100]:
# 評価
score = evaluate_answer_accuracy(question, response, gold_answer)
print(score)

2.0


### チャンク数を減らす
マッチした文を中心に前1文、後1文を含む計3文程度のチャンクを構成してみる。

In [101]:
# 前後それぞれ1つずつの文章を一つのドキュメントに追加する。（要は3つの文章集合になる)
references = "\n".join(["* " + "。".join(documents[max(0, i-1): min(i+1, len(documents))]).strip() for i in scores.argsort()[0][::-1][:topk]])
#query =  f"[参考資料]\n{references}\n\n[質問] LLMにおけるInference Time Scalingとは？"
query =  f"[参考資料]\n{references}\n\n[質問] tj-actions/changed-files の脆弱性によって、悪意あるコードがどのファイル／関数に追加されていたかを示し、この脆弱性により、どのような影響があったか、また、修正パッチではどのようにコードが書き換えられたかを説明してください。"
response = generate_output(query)
print(response)

## tj-actions/changed-files の脆弱性に関する説明

**脆弱性発生状況:**

* **発生時期:** 2025年3月12日から15日にかけて
* **原因:** tj-actions/changed-files のタグが全て悪意あるコミットに差し替えられ、回帰的に改ざんされた
* **影響範囲:** 
    * 23,000 以上のリポジトリで tj-actions/changed-files が使用されている
    * 公開ビルドログを持つプロジェクトは特にリスクが高い

**悪意のあるコードの追加:**

* **悪意あるGist取得・実行処理:** index.js から削除
* **外部スクリプト読み込み禁止:** action.yml で禁止

**影響:**

* **シークレット漏洩:** 3月14日20:30UTC 複数の公開リポジトリでシークレット漏洩が確認された。
* **外部接続監視:**  Falco ActionsやHarden-Runnerで外部接続を監視し、CI/CD ワークフローで gist.githubusercontent.com への不審な外部接続をリアルタイム検出するモジュールを導入。
* **ビルドログの書き出し:** AWSキー、GitHub PAT、npmトークン、RSA鍵などをビルドログに書き出した。

**修正内容:**

* **index.js から悪意あるGist取得・実行処理を削除:**  悪意のあるコードの追加を防ぐ
* **action.yml で外部スクリプト読み込みを禁止:**  外部スクリプトの読み込みを制限
* **全タグを安全なコミットに再ポイント:**  安全な状態に戻す
* **v46.0.1以降をリリース:**  脆弱性対策を施したバージョンをリリース


**コードの書き換え:**

* **index.js から悪意あるGist取得・実行処理を削除:**  悪意のあるコードの追加を防ぐ
* **action.yml で外部スクリプト読み込みを禁止:**  外部スクリプトの読み込みを制限
* **全タグを安全なコミットに再ポイント:**  安全な状態に戻す



**まとめ:**

tj-actions/changed-files の脆弱性により、悪意の

In [102]:
# 評価
score = evaluate_answer_accuracy(question, response, gold_answer)
print(score)

2.0


## 結果 (文脈付きチャンク化によるRAG)

文脈を含むチャンク化をしたが、チャンク数によるが、モデルの回答の方向性に明確な改善が見られました、チャンク数が少ない方がRAGで与えた情報を網羅する幅広い情報の出力になった。

#### チャンク数2
* gpt-4oによる評価：2.0  
脆弱性情報に、修正内容が混入。具体的な漏洩データが、修正内容として回答している。

#### チャンク数3
* gpt-4oによる評価：2.0  
脆弱性内容が細かく分類されたが、内容としては、チャンク数2の時と同様で、脆弱性情報に、修正内容が混入。具体的な漏洩データが、修正内容として回答している。

#### チャンク数1
* gpt-4oによる評価：2.0  
脆弱性の影響が出たタイムラインから始まっており、具体的な漏洩データが、影響として回答されるようになった。修正内容も正確。一方で、悪意のあるコードの追加、コードの書き換えの回答として、冗長感あり。

**今回の実験では、この出力が主観的に最もフィットする。**


### 残存する問題点
* チャンク数が多いと、質問と関連性の低い情報（ノイズ）が混入する

### 問題分析

文脈付きチャンク化によるアプローチで新たに発生した課題：

1. **情報過多の問題**:
   * ドキュメント量の増加により、モデルに提供される情報総量が大幅に増加
   * 関連情報と非関連情報が混在し、ノイズと重要情報の区別が困難に

2. **情報選択の複雑化**:
   * モデルは単に回答を生成するだけでなく、提供された多様な情報源から関連性の高い情報を選別する作業も担うことになった
   * この二重タスクにより回答生成の難易度が上昇




# 4. Rerankによる情報品質の向上

検索精度をさらに向上させるため、二段階の検索プロセスを導入します：

* **Rerank手法の導入**:
  - 第一段階: 従来通り基本的な検索アルゴリズムでtop-k個のドキュメントチャンクを取得
  - 第二段階: 取得したチャンクに対してLLMを活用した高度な関連性評価を実施
  - LLMに「このドキュメントは質問『tj-actions/changed-files の脆弱性によって、悪意あるコードがどのファイル／関数に追加されていたかを示し、修正パッチではどのようにコードが書き換えられたか？』に対して本当に関連性が高いか」を判断させる
  - 関連性スコアに基づいてランク付けし、真に関連性の高いチャンクのみを選出

* **期待される効果**:
  - 質の高い情報に焦点を絞ることで、ノイズとなる情報を大幅に削減
  - 文脈を保ちながらも、関連性の高い情報のみをモデルに提供
  - モデルのタスクを「多量の情報から選別して回答」から「厳選された情報に基づいて回答」へと単純化

この高度な情報フィルタリングにより、tj-actions/changed-files の脆弱性に関する正確で一貫性のある回答生成が期待されます。

In [103]:
references = []
for ref in ["。".join(documents[max(0, i-1): min(i+1, len(documents))]).strip() for i in scores.argsort()[0][::-1][:topk]]:
#for ref in ["。".join(documents[max(0, i-2): min(i+2, len(documents))]).strip() for i in scores.argsort()[0][::-1][:topk]]:
#for ref in ["。".join(documents[max(0, i-3): min(i+3, len(documents))]).strip() for i in scores.argsort()[0][::-1][:topk]]:

  #query = f"与えられた[参考資料]が[質問]に直接関連しているかを、'yes''no'で答えること。[参考資料]\n{ref}\n\n[質問] LLMにおけるInference Time Scalingとは？"
  query = f"与えられた[参考資料]が[質問]に直接関連しているかを、'yes''no'で答えること。[参考資料]\n{ref}\n\n[質問] tj-actions/changed-files の脆弱性によって、悪意あるコードがどのファイル／関数に追加されていたかを示し、この脆弱性により、どのような影響があったか、また、修正パッチではどのようにコードが書き換えられたかを説明してください。"

  response = generate_output(query)

  print("\n\n対象となるドキュメント:\n", ref.replace("。", "。\n"))
  print("\n関連しているかどうか: ", response)

  if "yes" in response.lower():
    references.append(ref)



対象となるドキュメント:
 脆弱性発生タイムラインは、2025年3月12日から15日にかけて tj-actions/changed-files のタグが全て悪意あるコミットに差し替えられ、回帰的に改ざんされたことがCISAからのアラートとして発出

関連しているかどうか:  yes 





対象となるドキュメント:
 公式アドバイザリは、CVE-2025-30066、CVE-2025-30154、GHSA-MRRH-FWG8-R2C3、CISAアラート「Supply Chain Compromise of Third-Party tj-actions/changed-files」（2025-03-18公開）。
影響範囲は、23,000 以上のリポジトリで tj-actions/changed-files が使用されており、公開ビルドログを持つプロジェクトは特にリスクが高い

関連しているかどうか:  no 





対象となるドキュメント:
 ログからBase64二重エンコード文字列を抽出して、出力内に AKIA… や ghp_… などのシークレットパターンが含まれていないかを検査する。
修正内容は、悪意あるGist取得・実行処理をindex.jsから削除し、action.ymlで外部スクリプト読み込みを禁止、全タグを安全なコミットに再ポイントしてv46.0.1以降をリリースした

関連しているかどうか:  no 





対象となるドキュメント:
 3月14日20:30UTC 複数の公開リポジトリでシークレット漏洩が確認されていることを、Aqua Security がブログで速報。
3月15日 GitHubが影響を受けた Action レポジトリを一時的に非公開化し、3月17日に悪意あるコードが除去されたパッチバージョン v46.0.1 がリリースされた

関連しているかどうか:  yes 





対象となるドキュメント:
 この時、AWSキー、GitHub PAT、npmトークン、RSA鍵などをビルドログに書き出した。
検出・診断方法は、GitHubコード検索で tj-actions/changed-files の使用箇所を特定し、Falco ActionsやHarden-Runnerで外部接続を監視し、CI/CD ワークフローで gist.gi

In [104]:
print(len(references))

3


上記より、上位3件のみが関連しているとわかったので、これらだけをモデルに渡すこととする。（生成内容が確立的なので、3件でない可能性もあります）

In [105]:
#query =  f"[参考資料]\n{references}\n\n[質問] LLMにおけるInference Time Scalingとは？"
query =  f"[参考資料]\n{references}\n\n[質問] tj-actions/changed-files の脆弱性によって、悪意あるコードがどのファイル／関数に追加されていたかを示し、この脆弱性により、どのような影響があったか、また、修正パッチではどのようにコードが書き換えられたかを説明してください。"
response = generate_output(query)
print(response)

## tj-actions/changed-files の脆弱性に関する情報

**脆弱性発生:** 2025年3月12日から15日にかけて、`tj-actions/changed-files` タグが全て悪意あるコミットに差し替えられ、回帰的に改ざんされたことがCISAからのアラートとして発出されました。

**影響:**

* **公開リポジトリのシークレット漏洩:** 3月14日、Aqua Securityがブログで速報し、複数の公開リポジトリでシークレット漏洩が確認されました。
* **GitHub Action レポジトリの非公開化:** 3月15日、GitHubは影響を受けた Action レポジトリを一時的に非公開化しました。
* **悪意あるコードの追加:**  脆弱性により、AWSキー、GitHub PAT、npmトークン、RSA鍵などをビルドログに書き出した。
* **外部接続の監視:**  Falco ActionsやHarden-Runnerで外部接続を監視し、CI/CD ワークフローで `gist.githubusercontent.com` への不審な外部接続をリアルタイム検出するモジュールを導入しました。

**悪意あるコードのファイル／関数の追加:**  具体的なファイルや関数の追加内容については、公開情報が限られています。

**修正パッチのコード書き換え:**  修正パッチでは、`tj-actions/changed-files` タグの悪意あるコミットから削除されたコードを削除し、安全なコードを導入しました。


**補足:**  現状の情報では、具体的な悪意あるコードの内容や書き換えの詳細が明らかになっていません。 






In [76]:
# 評価
score = evaluate_answer_accuracy(question, response, gold_answer)
print(score)

2.0


## 結果 (Rerank導入後)

Rerankの導入により、回答品質に改善が見られました：

* 回答出力が全体として冗長感はなくコンパクトになり、回答としては過不足ない。
* 一方で、悪意あるコードの追加に、漏洩データが記載されてしまっている。
